<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/ROC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd

# CoinGecko API'sinden veri almak için URL
api_url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin,ethereum,binancecoin,solana,cardano,fetch-ai&vs_currencies=usd"

# API'den veri alalım
response = requests.get(api_url)
data = response.json()

# Coin'lerin fiyat verilerini çıkaralım (Binance coin (BNB) da eklendi)
coins = {
    'BTC': {'current_price': data['bitcoin']['usd'], 'previous_price': 105851},  # Önceki fiyatı manuel olarak verebiliriz
    'ETH': {'current_price': data['ethereum']['usd'], 'previous_price': 3329.11},
    'BNB': {'current_price': data['binancecoin']['usd'], 'previous_price': 350.22},  # Binance coin (BNB)
    'SOL': {'current_price': data['solana']['usd'], 'previous_price': 270.39},
    'ADA': {'current_price': data['cardano']['usd'], 'previous_price': 0.975145},
    'FET': {'current_price': data['fetch-ai']['usd'], 'previous_price': 1.34}
}

# ROC hesaplamak için fonksiyon
def calculate_roc(current_price, previous_price):
    return ((current_price - previous_price) / previous_price) * 100

# Sonuçları saklamak için bir liste
roc_values = {}

# Her coin için ROC hesaplayalım
for coin, prices in coins.items():
    roc_values[coin] = calculate_roc(prices['current_price'], prices['previous_price'])

# Sonuçları DataFrame olarak gösterebiliriz
roc_df = pd.DataFrame(list(roc_values.items()), columns=['Coin', 'ROC (%)'])
roc_df['ROC (%)'] = roc_df['ROC (%)'].round(2)  # Yüzdeyi iki ondalıklı olarak yuvarlayalım

# BTC'nin ROC değerini aşan coin'leri bulalım
btc_roc = roc_values['BTC']
coins_above_btc_roc = roc_df[roc_df['ROC (%)'] > btc_roc]

# Sonuçları yazdıralım
print("Tüm Coin'lerin ROC Değerleri:")
print(roc_df)
print("\nBTC ROC Değerini Geçen Coin'ler:")
print(coins_above_btc_roc)


Tüm Coin'lerin ROC Değerleri:
  Coin  ROC (%)
0  BTC    -2.91
1  ETH    -3.41
2  BNB    96.16
3  SOL    -7.73
4  ADA    -0.43
5  FET    -7.46

BTC ROC Değerini Geçen Coin'ler:
  Coin  ROC (%)
2  BNB    96.16
4  ADA    -0.43
